# aspects


Aspects provide additional functionality to the classic Object-Oriented pattern.

https://en.wikipedia.org/wiki/Aspect-oriented_programming



## introduction


Types of aspects:

- affect - run after the code
- prefect - run before the code
- inflect - make available part of the affection within the code
- reflect - describe the current affection
- infect - replace part of code with result of another
- effect - return a list of affections the occured or will occur if reference is provided
- expect - deriving from the context find the variables involved and if they do not meet expectation create a "soft assertion"?
- inspect - show debugging information surrounding the specified code


TODO: aspect babel plugin. This will use the transpiler.

TODO: depends on transpiling kernels


## transpiler bindings

Transpile calls that bind functionality to the neessary directives.

TODO: inspect, find calls to aspect and interpret them up front.



### inspect

Logging is one of the most obvious use cases for aspect oriented programming. It makes sense to start with this.  Log messages are inserted at every possible point in the program execution. When transpiled, additional sum information is collected and provided at the end, useful for attaching a unit test and code coverage reporter.



#### the code

inspect every statement?


In [ ]:
var importer = require('../Core')
var {transpile} = importer.import('transpile code')
var {selectAst, makeExpr} = importer.import('select code tree')
var {htmlToTree} = importer.import('html to tree')

var STATEMENTS = `//*[contains(@type, "Declaration")]
|//*[contains(@type, "Statement")]`
var NEAR_IDENTIFIERS = `
 ./Identifier/@name
|./*/Identifier/@name
|./*/*/Identifier/@name
|./*/*/*/Identifier/@name

|./parent::*/Identifier/@name
|./parent::*/*/Identifier/@name
|./parent::*/*/*/Identifier/@name
|./parent::*/*/*/*/Identifier/@name

|./parent::*/parent::*/Identifier/@name
|./parent::*/parent::*/*/Identifier/@name
|./parent::*/parent::*/*/*/Identifier/@name
|./parent::*/parent::*/*/*/*/Identifier/@name

`

function inspectCallback(ctx) {
    console.log(JSON.stringify(ctx))
}

function inspectTemplate(ctx) {
    // code inserted in to transpiled module
    inspectCallback(ctx)
}

function insertInspect(filename, code, ctx) {
    var inspect = makeExpr(inspectTemplate)

    // replace line with the line number from original range
    var range = JSON.parse(ctx.getAttribute('range'))
    var line = code.substr(0, range[0]).split('\n').length
    
    // replace the ctx with nearby identifiers
    // TODO: 'replace' transpiler command
    var replaceCtx = selectAst(`.//Identifier[@name="ctx"]`, inspect)
    var nearbyIdentifiers = selectAst([NEAR_IDENTIFIERS], ctx)
    var nearbyCtx = makeExpr(`{
${nearbyIdentifiers.join(',')},
//ctx: ${JSON.stringify(htmlToTree(ctx))},
type: "${ctx.getAttribute('type')}",
line: ${line},
filename: "${filename}"
}`)
    nearbyCtx.setAttribute('parent-attr', 'arguments')
    replaceCtx.replaceWith(nearbyCtx)
    
    // insert into parent statement body
    // TODO: make this a transpile operation because it contains node calls
    var parent = selectAst(`./parent::*`, ctx)
    Array.from(inspect.childNodes).forEach(n => {
        if(n.setAttribute) {
            n.setAttribute('parent-attr', 'body')
        }
        parent.insertBefore(n, ctx)
    })
}

function transpileInspect(code, filename) {
    return transpile([
        [STATEMENTS, insertInspect.bind(null, filename, code)]
    ], code)
}

module.exports = {
    inspectCallback,
    inspectTemplate,
    transpileInspect,
}


#### test statement inspector



In [ ]:
var importer = require('../Core')
var {transpileInspect} = importer.import('inspect every statement')

var code = `
var importer = require('../Core');

function addImports() {

}

`

if(typeof $$ != 'undefined') {
    $$.mime({'text/plain': transpileInspect(code, 'test_code.js')})
    
    /*
    expected output 
var addImports = importer.import('add missing imports')
*/
    
}
